In [32]:
import fastf1
import pandas as pd

session = fastf1.get_session(2025, 'Abu Dhabi', 'R')
session.load(telemetry = False, weather = False)

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No 

In [33]:
laps = session.laps
drivers = session.drivers

In [34]:
pos_data = laps[['LapNumber', 'Driver', 'Position', 'PitInTime']].copy()
pos_data = pos_data.dropna(subset = ['Position'])
pos_data['Pitted'] = ~pd.isna(pos_data['PitInTime'])

lap_positions = {}
lap_pitted = {}

In [35]:
max_lap = int(pos_data['LapNumber'].max())
for lap in range(1, max_lap + 1):
    current_lap_data = pos_data[pos_data['LapNumber']==lap]
    lap_positions[lap] = dict(zip(current_lap_data['Driver'], current_lap_data['Position']))

    lap_pitted[lap] = dict(zip(current_lap_data['Driver'], current_lap_data['Pitted']))
overtakes = []

In [36]:
for lap in range(2, max_lap+1):
    prev_lap_map = lap_positions.get(lap-1, {})
    curr_lap_map = lap_positions.get(lap, {})
    pitted_map = lap_pitted.get(lap, {})

    common_drivers = set(prev_lap_map.keys()) & set(curr_lap_map.keys())
    common_drivers = list(common_drivers)

    for i in range(len(common_drivers)):
        for j in range(i+1, len(common_drivers)):
            d1 = common_drivers[i]
            d2 = common_drivers[j]

            d1_prev = prev_lap_map[d1]
            d2_prev = prev_lap_map[d2]

            d1_curr = curr_lap_map[d1]
            d2_curr = curr_lap_map[d2]

            if (d1_prev > d2_prev) and (d1_curr < d2_curr):
                overtaker = d1
                victim = d2
            elif (d2_prev > d1_prev) and (d2_curr < d1_curr):
                overtaker = d2
                victim = d1
            else:
                continue

            is_pit_pass = pitted_map.get(victim, False)

            overtakes.append({
                'Lap': lap,
                'Overtaker': overtaker,
                'Overtaken': victim,
                'Type': 'Pit Stop Pass' if is_pit_pass else 'On Track Overtake'
            })

In [37]:
overtakes_df = pd.DataFrame(overtakes)
print(f"Total Position Changes Found: {len(overtakes_df)}")
print(f"On-Track Overtakes: {len(overtakes_df[overtakes_df['Type']== 'On Track Overtake'])}")
print("First 10 overtakes:")
print(overtakes_df.head(10))

Total Position Changes Found: 242
On-Track Overtakes: 215
First 10 overtakes:
   Lap Overtaker Overtaken               Type
0    2       HAM       SAI  On Track Overtake
1    4       RUS       ALO  On Track Overtake
2    8       STR       ALB      Pit Stop Pass
3    8       ANT       ALB      Pit Stop Pass
4    8       SAI       HAM      Pit Stop Pass
5    8       COL       HUL  On Track Overtake
6    8       GAS       HUL  On Track Overtake
7    9       COL       ALB  On Track Overtake
8    9       HUL       ALB  On Track Overtake
9    9       GAS       ALB  On Track Overtake


In [38]:
import plotly.express as px

viz_data = laps[['Driver', 'LapNumber', 'Position', 'Team']].dropna(subset = ['Position'])
viz_data = viz_data[viz_data['LapNumber'] <= 72]

race_name = session.event['EventName']
race_year = session.event.year

fig = px.line(
    viz_data,
    x = 'LapNumber',
    y = 'Position',
    color = 'Driver',
    title = f'{race_year} {race_name} - Position Changes',
    markers = True,
    template = 'plotly_dark',
    hover_data = ['Team']
)

fig['layout']['yaxis']['autorange'] = 'reversed'

fig.show()